# Intermediate Mancala Progress
**Jenn Kaphammer**

## Approach
### Current libraries utilized
- random
- pandas
- Counter
### Methods
- Adapting minimax and Alpha-beta Pruning algorithms from the AIMA-Python github

## Progress/results so far
### Progress
- Executed 100 games of Rand vs. Rand player and analyzed results
- Updated utility function in Mancala DS
- Begain altering AIMA minimax function to fit mancala
  - This does not work with the mancala yet but I am currently mapping out on paper how to alter this function

## Previously established code
This code features the Mancala data structure and functions usable to play the game. It also features a 'control' random selection player.\
**I added a winner attribute on the Mancala data strucutre and a get winner function to more easily analyze win results.** \
**The utility funciton of stones in Max Mancala - # stone in Min Mancala has also been added to this**

In [64]:
import random
import pandas as pd
random.seed(109)

In [138]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

        #adding winner
        self.winner = None

    def utility(self, player):
        if player == 1:
            player_stones = self.board[self.p1_mancala_index]
            opponent_stones = self.board[self.p2_mancala_index]
        
        elif player == 2:
            player_stones = self.board[self.p2_mancala_index]
            opponent_stones = self.board[self.p1_mancala_index]
        
        utility = player_stones - opponent_stones
        return utility
        
    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        #if self = player 1
            # must pick from p1 pits index
                #IF NOT: break, Flag
                #after in p1 index: pit chosen must not be empty
                    #IF NOT: Flag
        # elif self = p2
            #must pick from p2 pits index
                #IF NOT: break, Flag
                #after in p2 index: pit chosen must not be empty
                    #IF NOT: break, Flag
        #IF FLAG:
            #print "Invalid Move"
            #return false
        #else
            #return true
        # write your code here
        flag = 0
        if self.current_player == 1:
            if pit < 1 or pit > self.pits_per_player:
                flag = 1
            else:
                selectedPit = self.p1_pits_index[0] + pit - 1
            if self.board[selectedPit] == 0:
                flag = 1
        elif self.current_player == 2:
            if pit < 1 or pit > self.pits_per_player:
                flag = 1
            else:
                selectedPit = self.p2_pits_index[0] + pit - 1
            if self.board[selectedPit] == 0:
                flag = 1
        if flag == 1:
            print("Invalid move")
            return False
        return True
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        #PART 2
        #generate random int between 1-pits per player
        #apply it to whichever players turn it is
        #if it is empty regenerate
        # write your code here
        while True:
            pit = random.randint(1, self.pits_per_player)
            if self.valid_move(pit):
                break
        return pit
    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        
        #check = validmove(self, currpit)
        #continue if validmove(currpit) = false

        #win = winning_eval(self)
            #if win == true, terminate
        
        #isolate pit
        #iterate stones until no more stones to distribute
        #if stone lands in curr player empty pit
            #place into curr player mancala
            #if inverse pit has stones
                #place into curr player mancala
        
        # write your code here
        checkWin = self.winning_eval()
        if checkWin == True:
            # self.winning_eval()
            return -2
        print(f"Player {self.current_player} chose pit: {pit}")
        
        checkMove = self.valid_move(pit)
        if checkMove == False:
            return -1
        self.moves.append((self.current_player, pit))
            
        #if both checks passed
        stoneCounter = 0

        curr = pit
        if self.current_player == 1:
            #move stones
            selectedPit = self.p1_pits_index[0] + pit - 1
            stoneCounter = self.board[selectedPit]
            #set stones in this pit to 0
            self.board[selectedPit] = 0
            curr = selectedPit
            while stoneCounter > 0:
                curr = (curr+1)%len(self.board)
                #skip opponents mancala
                if curr == self.p2_mancala_index:
                    curr = (curr + 1) % len(self.board)
                self.board[curr]+=1
                stoneCounter -= 1
            #if needing to add to mancala
            if self.board[curr] == 1:
                inversePit = abs(curr - (2 * self.pits_per_player))
                if curr in range(self.p1_pits_index[0],self.p1_pits_index[1]+1):
                    self.board[self.p1_mancala_index] += self.board[inversePit] + 1
                    self.board[inversePit] = 0
                    self.board[curr] = 0            
            self.current_player = 2
            
        elif self.current_player == 2:
            #move stones
            selectedPit = self.p2_pits_index[0] + pit - 1
            stoneCounter = self.board[selectedPit]
            self.board[selectedPit] = 0
            curr = selectedPit
            while stoneCounter > 0:
                curr = (curr+1)%len(self.board)
                if curr == self.p1_mancala_index:
                    curr = (curr + 1) % len(self.board)
                self.board[curr]+=1
                stoneCounter -= 1
            #if needing to add to mancala
            if self.board[curr] == 1:
                inversePit = abs(curr - (2 * self.pits_per_player))
                if curr in range(self.p2_pits_index[0],self.p2_pits_index[1]+1):
                    self.board[self.p2_mancala_index] += self.board[inversePit] + 1
                    self.board[inversePit] = 0
                    self.board[curr] = 0  
            self.current_player = 1
        
        return self.board
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        #not won : False
        #game over : True
        
        # loop through p1 pits
            # if any pit has any amount of stones
                #return False
        # loop through p2 pits
            # if any pit has any amount of stones
                #return False
        #return True
        #The winner is the player with the most stones in their mancala. 
        #If both mancalas have the same number of stones, it's a tie.

        # write your code here
        p1Empty = all(stones == 0 for stones in self.board[self.p1_pits_index[0]: self.p1_pits_index[1] + 1])
        p2Empty = all(stones == 0 for stones in self.board[self.p2_pits_index[0]: self.p2_pits_index[1] + 1])
        
        if p1Empty or p2Empty:
            print("Game Over")
            #determine whos mancala is bigger
            if self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]:
                self.winner = 1
                print("Player 1 won")
            elif self.board[self.p1_mancala_index] < self.board[self.p2_mancala_index]:
                self.winner = 2
                print("Player 2 won")
            else:
                self.winner = 0
                print("Tie")
            return True
        return False
    def get_winner(self):
        return self.winner


## Random player testing
### Playing 100 Games of Random Player vs. Random Player
**See code below for these games.**
### RESULTS:
- What percentage of games does each player (1st or 2nd) win?
    - From my simulations, player 1 won 51% of the time, player 2 won 45% of the time, and there was a tie 5% of the time.
- On average, how many moves does it take to win?
    - It took on average 41.88 moves for the game to conclude, so about 42 moves.

In [108]:
#Play a game with random players
def singleRandomPlayerGame(pits_per_player=6, stones_per_pit=4):
    game = Mancala(pits_per_player, stones_per_pit)
    moveCounter = 0
    # invalidMoveCounter = 0

    while True:
        #if game is already over
        if game.winning_eval():
            break
        pit = game.random_move_generator()
        choose = game.play(pit)
        
        if choose == -2:
            break
            #winning flag
        elif choose == -1:
            continue
            # if invalid move

        moveCounter += 1
    winner = game.get_winner()
    return winner, moveCounter

In [110]:
#run simulations and put results in dataframe
def runRandSimulations(numGames = 100, pits_per_player=6, stones_per_pit=4):
    results = []
    for _ in range(numGames):
        winner, moveCounter = singleRandomPlayerGame(pits_per_player, stones_per_pit)
        results.append({'winner': winner, 'moves': moveCounter})
    
    randPlayers = pd.DataFrame(results)
    return randPlayers

In [112]:
#Running the actual games
randResults = runRandSimulations(100, 6, 4)
randResults

Player 1 chose pit: 1
Player 2 chose pit: 5
Player 1 chose pit: 3
Player 2 chose pit: 2
Player 1 chose pit: 4
Player 2 chose pit: 6
Player 1 chose pit: 2
Player 2 chose pit: 5
Player 1 chose pit: 5
Player 2 chose pit: 3
Player 1 chose pit: 6
Player 2 chose pit: 1
Invalid move
Player 1 chose pit: 4
Player 2 chose pit: 4
Player 1 chose pit: 3
Player 2 chose pit: 6
Player 1 chose pit: 2
Invalid move
Player 2 chose pit: 2
Player 1 chose pit: 6
Player 2 chose pit: 4
Invalid move
Player 1 chose pit: 3
Invalid move
Player 2 chose pit: 1
Invalid move
Invalid move
Invalid move
Invalid move
Invalid move
Invalid move
Invalid move
Invalid move
Player 1 chose pit: 5
Player 2 chose pit: 2
Player 1 chose pit: 6
Player 2 chose pit: 4
Player 1 chose pit: 4
Invalid move
Player 2 chose pit: 3
Player 1 chose pit: 6
Player 2 chose pit: 4
Player 1 chose pit: 5
Player 2 chose pit: 5
Player 1 chose pit: 2
Player 2 chose pit: 1
Player 1 chose pit: 3
Invalid move
Player 2 chose pit: 6
Player 1 chose pit: 1
Inva

,winner,moves
0,2,46
1,2,61
2,1,39
3,1,47
4,1,38
...,...,...
95,2,27
96,2,47
97,2,29
98,2,23


In [127]:
#percentage of each player winning
winningPercentage = randResults['winner'].value_counts(normalize=True) * 100
print(winningPercentage)

winner
1    51.0
2    45.0
0     4.0
Name: proportion, dtype: float64


In [120]:
#average amount of moves taken to win
avgMoves = randResults['moves'].mean()
print(avgMoves)

41.88


## AI Player that utilizes minimax
### Adapted from the AIMA-Python library

In [ ]:
def minmax_decision(state, game):
    """Given a state in a game, calculate the best move by searching
    forward all the way to the terminal states. [Figure 5.3]"""

    player = game.to_move(state)

    def max_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v

    def min_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v

    # Body of minmax_decision:
    return max(game.actions(state), key=lambda a: min_value(game.result(state, a)))